In [1]:
# jupyter notebook
#!jupyter nbextension enable --py --sys-prefix ipyleaflet

In [2]:
# jupyter lab
# !jupyter labextension install jupyter-leaflet
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [3]:
import os
import sys
import time
import pickle
import string
from datetime import datetime
from datetime import timedelta
from itertools import chain, cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib

import numpy as np
import pandas as pd
from pandas.tools.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

#from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg
from geopy.geocoders import Nominatim
import geopandas as gpd
import ipyleaflet
#from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, WMSLayer, Marker, MarkerCluster

##https://python-visualization.github.io/folium/docs-v0.6.0/
#https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf
import folium
import folium.plugins
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster


#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python

In [4]:
pwd

'C:\\Users\\rsouza\\Documents\\Python Scripts\\exploreAT-Topothek.NLP'

#### Reading the XML Metadata files and processing Information on places

In [5]:
placesfile =  './TEI-XML-2018/helper_tables/listPlace.xml'
#print((open(placesfile, encoding="utf-8").read()))
placestree = ET.parse(placesfile)
placesroot = placestree.getroot()
set([elem.tag for elem in placesroot.iter()])

{'{http://www.tei-c.org/ns/1.0}Bundesland',
 '{http://www.tei-c.org/ns/1.0}Gemeinde',
 '{http://www.tei-c.org/ns/1.0}Großregion',
 '{http://www.tei-c.org/ns/1.0}Kleinregion',
 '{http://www.tei-c.org/ns/1.0}Kommentar',
 '{http://www.tei-c.org/ns/1.0}Ort',
 '{http://www.tei-c.org/ns/1.0}Sigle_DB',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}note',
 '{http://www.tei-c.org/ns/1.0}place',
 '{http://www.tei-c.org/ns/1.0}placeName',
 '{http://www.tei-c.org/ns/1.0}record'}

In [6]:
placesdict = xmltodict.parse((open(placesfile, encoding="utf-8").read()))
print(placesdict.keys())
print(placesdict['listPlace'].keys())

odict_keys(['listPlace'])
odict_keys(['@xmlns', 'place'])


In [7]:
placeslist = placesdict['listPlace']['place']
state_bundesland = [p['placeName'] for p in placeslist]
print(state_bundesland)

['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.']


In [8]:
placeslist[0]['listPlace']['place']['listPlace']['place']['listPlace']['place']['listPlace']['place'][0]['record']
#placeslist[1]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['record']
#placeslist[2]['listPlace']['place']['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['record']
#placeslist[3]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['listPlace']['place']['record']

OrderedDict([('Ort', 'Altmannsdorf'),
             ('Gemeinde', 'Wien'),
             ('Kleinregion', 'W.'),
             ('Großregion', 'W.'),
             ('Bundesland', 'W.'),
             ('Sigle_DB', None),
             ('Kommentar', None)])

In [9]:
identified_places = collections.defaultdict(list)
def show_record(something):
    if isinstance(something, list):
        for elem in something:
            show_record(elem)
    elif isinstance(something, collections.OrderedDict) and ('record' in something.keys()):
        try:
            #print(something['record']['Bundesland'], ' --> ', something['record']['Ort'])
            identified_places[something['record']['Bundesland']].append(something['record']['Ort'])
        except:
            #print(something['record'][0]['Bundesland'], ' --> ', something['record'][0]['Ort'])
            identified_places[something['record'][0]['Bundesland']].append(something['record'][0]['Ort'])
    elif isinstance(something, collections.OrderedDict):
        new = something['listPlace']['place']
        show_record(new)
    else:
        print(type(something))
        print(something)

In [10]:
show_record(placeslist)

In [11]:
identified_places.keys()

dict_keys(['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.'])

In [25]:
df_places = pd.DataFrame(columns=['Bundesland', 'Ort'])
for key in identified_places.keys():
    df_temp = pd.DataFrame(identified_places[key], columns=['Ort'])
    df_temp['Bundesland'] = key
    df_places = pd.concat([df_places, df_temp], axis=0, sort=True)
df_places.head(30)

,Bundesland,Ort
0,W.,Altmannsdorf
1,W.,Aspern
2,W.,Atzgersdorf
3,W.,Breitensee
4,W.,Favoriten
5,W.,Gersthof
6,W.,Hernals
7,W.,Hietzing
8,W.,Inzersdorf
9,W.,Kalksburg


In [49]:
df_places.reset_index(inplace=True, drop=True)

In [71]:
df_places.replace('W.','Wien', inplace=True)
df_places.replace('STir.','Südtirol', inplace=True)
df_places.replace('OTir.','Osttiroler', inplace=True)
df_places.replace('NTir.','Nord tirol', inplace=True)
df_places.replace('Kä.','Kärnten', inplace=True)
df_places.replace('St.','Steiermark', inplace=True)
df_places.replace('Sa.','Salzburg', inplace=True)
df_places.replace('OÖ','Oberösterreich', inplace=True)
df_places.replace('NÖ','Niederösterreich', inplace=True)
df_places.replace('Bgl.','Burgenland', inplace=True)

In [39]:
df_places.head(10)

,index,Bundesland,Ort
0,0,Wien,Altmannsdorf
1,1,Wien,Aspern
2,2,Wien,Atzgersdorf
3,3,Wien,Breitensee
4,4,Wien,Favoriten
5,5,Wien,Gersthof
6,6,Wien,Hernals
7,7,Wien,Hietzing
8,8,Wien,Inzersdorf
9,9,Wien,Kalksburg


In [41]:
df_places.loc[0].values

array([0, 'Wien', 'Altmannsdorf'], dtype=object)

#### Using a geolocator to get the Coordinates

In [31]:
geolocator = gg.Nominatim(user_agent="test_application")

In [43]:
df_places.loc[0, 'Ort'] + ', ' +  df_places.loc[0, 'Bundesland']

'Altmannsdorf, Wien'

In [44]:
logradouro = df_places.loc[0, 'Ort'] + ', ' +  df_places.loc[0, 'Bundesland']
location = geolocator.geocode(logradouro)
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

Altmannsdorf, Hoffingergasse, KG Altmannsdorf, Meidling, Kabelwerk, Wien, 1120, Österreich
(48.16443205, 16.3217472203092)
{'place_id': '157414934', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': '366749912', 'boundingbox': ['48.1554092', '48.1735359', '16.304118', '16.3372646'], 'lat': '48.16443205', 'lon': '16.3217472203092', 'display_name': 'Altmannsdorf, Hoffingergasse, KG Altmannsdorf, Meidling, Kabelwerk, Wien, 1120, Österreich', 'class': 'boundary', 'type': 'statistical', 'importance': 0.48123930027933803}


The next cells were used just for getting geographic information. The saved Pickle shall be used for the subsequent processing. 

In [45]:
def get_coordinates(place):
    try:
        location = geolocator.geocode(place)
    except:
        return None
    print(location)
    time.sleep(5)
    if location:
        return (location.longitude, location.latitude), location.address
    else:
        return None

In [46]:
geoinfo = []
for idx in df_places.index:
    geoinfo.append(get_coordinates(df_places.loc[idx, 'Ort'] + ', ' +  df_places.loc[idx, 'Bundesland']))

Altmannsdorf, Hoffingergasse, KG Altmannsdorf, Meidling, Kabelwerk, Wien, 1120, Österreich
Aspern, Donaustadt, Wien, 1220, Österreich
Atzgersdorf, Liesing, Wien, 1230, Österreich
Breitensee, Penzing, Wien, 1140, Österreich
Favoriten, Wien, 1100, Österreich
Gersthof, Währing, Wien, 1180, Österreich
Hernals, Wien, 1170, Österreich
Hietzing, Wien, 1130, Österreich
KG Inzersdorf, Liesing, Wien, 1230, Österreich
Kalksburg, Liesing, Wien, 1230, Österreich
Leopoldau, Floridsdorf, Wien, 1210, Österreich
Mauer, Liesing, Wien, 1230, Österreich
Meidling, Wien, 1120, Österreich
KG Ober St. Veit, Hietzing, Wien, 1130, Österreich
Oberdöbling, Döbling, Wien, 1190, Österreich
Sievering, Döbling, Wien, 1190, Österreich
Währing, Wien, 1180, Österreich
Wien, Österreich
Langtaufers: Pfraktion Pedross, Zerkaser - Casera, Graun - Curon Venosta, Vinschgau - Val Venosta, BZ, TAA, 39027, Italia
Via Claudia See, Reschenstraße, Pfunds, Landeck, Tirol, 6542, Österreich
St. Valentin auf der Haide - San Valentino a

Kurtinig an der Weinstraße - Cortina sulla Strada del Vino, Überetsch-Unterland - Oltradige-Bassa Atesina, BZ, TAA, Italia
Margreid an der Weinstraße - Magrè sulla Strada del Vino, Überetsch-Unterland - Oltradige-Bassa Atesina, BZ, TAA, Italia
Montan - Montagna, Auer - Ora, Überetsch-Unterland - Oltradige-Bassa Atesina, BZ, TAA, 39040, Italia
Egna - Neumarkt, 7, Rathausring - Largo Municipio, Neumarkt - Egna, Überetsch-Unterland - Oltradige-Bassa Atesina, BZ, TAA, 39044, Italia
Cauria - Gfrill, Überetsch-Unterland - Oltradige-Bassa Atesina, BZ, TAA, 38040, Italia
Salurn - Salorno, Via Stazione - Bahnhofstraße, Salorno - Salurn, Überetsch-Unterland - Oltradige-Bassa Atesina, BZ, TAA, 39040, Italia
Tramin an der Weinstraße - Termeno sulla Strada del Vino, Überetsch-Unterland - Oltradige-Bassa Atesina, BZ, TAA, Italia
1 - Truden-Cislon - Trodena-Cislon, Truden im Naturpark - Trodena nel Parco Naturale, Überetsch-Unterland - Oltradige-Bassa Atesina, BZ, TAA, 39040, Italia
2 - 1 - Katzenlei

Oberrasen - Rasun di Sopra, Pustertal - Val Pusteria, BZ, TAA, Italia
Handwerkerzone Rasen - Zona artigianale Rasun, Rauterhöfe - Masi Raut, Rasen-Antholz - Rasun-Anterselva, Pustertal - Val Pusteria, BZ, TAA, 39030, Italia
St. Martin - San Martino in Casies, Pustertal - Val Pusteria, BZ, TAA, Italia
Freiwillige Feuerwehr St. Martin/Gsies, 1/B, Gsieser Straße Via Casies, Steger Höfe, Gsies - Valle di Casies, Pustertal - Val Pusteria, BZ, TAA, 39030, Italia
Freiwillige Feuerwehr Prags - Vigili del Fuoco volontari Braies, 42, Innerprags - Braies di dentro, Prags - Braies, Pustertal - Val Pusteria, BZ, TAA, 39030, Italia
Innichberg, Innichen - San Candido, Pustertal - Val Pusteria, BZ, TAA, Italia
Vierschach, Innichen - San Candido, Pustertal - Val Pusteria, BZ, TAA, Italia
Winnebach, Terenten - Terento, Pustertal - Val Pusteria, BZ, TAA, Italia
Innichen, Innichen - San Candido, Pustertal - Val Pusteria, BZ, TAA, Italia
OK Südtirol Dolomiti Superbike, 6, Piazza Von Kurz - Von Kurz Platz, 

Ranggen, Innsbruck-Land, Tirol, 6179, Österreich
Rietz, Imst, Tirol, 6421, Österreich
Silz, Malchow, Mecklenburgische Seenplatte, Mecklenburg-Vorpommern, Deutschland
Stams, Imst, Tirol, 6422, Österreich
Telfs, Innsbruck-Land, Tirol, 6410, Österreich
Unterperfuss, Innsbruck-Land, Tirol, 6178, Österreich
Burgruine Fragenstein, B177, Thomasegg, Zirl, Innsbruck-Land, Tirol, 6170, Österreich
Zirl, Innsbruck-Land, Tirol, 6170, Österreich
Gurgl, Leutasch, Innsbruck-Land, Tirol, 6410, Österreich
Sölden, Imst, Tirol, Österreich
Huben, Hintertiefenbach, Herrstein, Landkreis Birkenfeld, Rheinland-Pfalz, 55743, Deutschland
Längenfeld, Imst, Tirol, 6444, Österreich
Parco degli olivi, Via delle Brae, Nago-Torbole, Comunità Alto Garda e Ledro, TN, TAA, 38069, Italia
Oetz, Imst, Tirol, 6433, Österreich
Sautens, Imst, Tirol, 6432, Österreich
Umhausen, Imst, Tirol, 6441, Österreich
Gries im Sellrain, Innsbruck-Land, Tirol, 6182, Österreich
Grinzens, Innsbruck-Land, Tirol, 6095, Österreich
Sellrain, Inns

Itter, Kitzbühel, Tirol, 6305, Österreich
None
Kirchberg in Tirol, Kitzbühel, Tirol, 6365, Österreich
Reith bei Kitzbühel, Kitzbühel, Tirol, Österreich
Westendorf, Kitzbühel, Tirol, 6363, Österreich
Ellmau, Kufstein, Tirol, 6352, Österreich
Scheffau am Wilden Kaiser, Kufstein, Tirol, 6351, Österreich
Söll, Kufstein, Tirol, 6306, Österreich
Kössen, Kitzbühel, Tirol, 6345, Österreich
Schwendt, Kitzbühel, Tirol, 6385, Österreich
Walchsee, Kufstein, Tirol, 6344, Österreich
Going am Wilden Kaiser, Kitzbühel, Tirol, 6353, Österreich
Erpfendorf, Kitzbühel, Tirol, 6383, Österreich
Kirchdorf in Tirol, Kitzbühel, Tirol, Österreich
Kitzbühel, Tirol, 6370, Österreich
Oberndorf in Tirol, Kitzbühel, Tirol, 6372, Österreich
St. Johann in Tirol, Kitzbühel, Tirol, 6380, Österreich
Aurach bei Kitzbühel, Kitzbühel, Tirol, 6371, Österreich
Jochberg, Schwangau, Landkreis Ostallgäu, Schwaben, Bayern, Deutschland
Fieberbrunn, Kitzbühel, Tirol, 6391, Österreich
Hochfilzen, Kitzbühel, Tirol, 6395, Österreich
S

...

Vitis, Bezirk Waidhofen an der Thaya, Niederösterreich, 3902, Österreich
Puch, Mas-Grenier, Montauban, Tarn-et-Garonne, Occitanie, France métropolitaine, 82600, France
Waidhofen an der Thaya, Bezirk Waidhofen an der Thaya, Niederösterreich, 3830, Österreich
Sarning, Bezirk Waidhofen an der Thaya, Niederösterreich, Österreich
Waidhofen an der Thaya-Land, Bezirk Waidhofen an der Thaya, Niederösterreich, Österreich
Fratres, Bezirk Waidhofen an der Thaya, Niederösterreich, 3844, Österreich
Waldkirchen an der Thaya, Bezirk Waidhofen an der Thaya, Niederösterreich, 3844, Österreich
Windigsteig, Bezirk Waidhofen an der Thaya, Niederösterreich, 3841, Österreich
Zissersdorf, Bezirk Horn, Niederösterreich, 2094, Österreich
Drosendorf, Bezirk Horn, Niederösterreich, 2095, Österreich
Goggitsch, Weiz, Steiermark, 8323, Österreich
Geras, Bezirk Horn, Niederösterreich, 2093, Österreich
Niederfladnitz, Bezirk Hollabrunn, Niederösterreich, 2081, Österreich
Hardegg, Bezirk Hollabrunn, Niederösterreich, 

Senftenberg, Oberspreewald-Lausitz, Brandenburg, Deutschland
Stratzing, Bezirk Krems, Niederösterreich, 3552, Österreich
Gedersdorf, Bezirk Krems, Niederösterreich, 3494, Österreich
Diendorf, Landkreis Schwandorf, OPf, Bayern, 92431, Deutschland
Engabrunn, Bezirk Krems, Niederösterreich, 3485, Österreich
None
Grafenegg, Bezirk Krems, Niederösterreich, Österreich
Hadersdorf, Kremser Straße, Gemeinde Hadersdorf-Kammern, Bezirk Krems, Niederösterreich, 3493, Österreich
Mittelberg, Tegeler Weg, Geismar, Göttingen, Landkreis Göttingen, Niedersachsen, 37085, Deutschland
Štítary, okres Znojmo, Jihomoravský kraj, Jihovýchod, 67102, Česko
Langenlois, Bezirk Krems, Niederösterreich, 3550, Österreich
Mollands, Bezirk Krems, Niederösterreich, 3562, Österreich
Schönberg am Kamp, Bezirk Krems, Niederösterreich, 3562, Österreich
1, Güterweg Elsarn, Rothmühle, Gemeinde Straß im Straßertale, Bezirk Krems, Niederösterreich, 3491, Österreich
Falkenberg, Kjølebrønn, Kragerø, Telemark, Norge
Gemeinde Straß

Hochleithen, Rotthalmünster (VGem), Landkreis Passau, NB, Bayern, 94094, Deutschland
Hohenruppersdorf, Bezirk Gänserndorf, Niederösterreich, 2223, Österreich
Jedenspeigen, Bezirk Gänserndorf, Niederösterreich, 2264, Österreich
Hautzendorf, Graz-Umgebung, Steiermark, 8141, Österreich
Kreuttal, Schleinbach, Gemeinde Kreuttal, Bezirk Korneuburg, Niederösterreich, 2112, Österreich
Kreuzstetten, Bezirk Mistelbach, Niederösterreich, Österreich
Ladendorf, Bezirk Mistelbach, Niederösterreich, 2126, Österreich
Lanzendorf, Landkreis Kulmbach, Oberfranken, Bayern, 95502, Deutschland
Mistelbach, Mistelbach (VGem), Landkreis Bayreuth, Oberfranken, Bayern, 95511, Deutschland
Niederleis, Bezirk Mistelbach, Niederösterreich, Österreich
Ringelsdorf-Niederabsdorf, Gemeinde Ringelsdorf-Niederabsdorf, Bezirk Gänserndorf, Niederösterreich, 2272, Österreich
Spannberg, Bezirk Gänserndorf, Niederösterreich, 2244, Österreich
Erdpreß, Bezirk Gänserndorf, Niederösterreich, 2224, Österreich
Sulz im Weinviertel, G

Gemeinde Donnerskirchen, Bezirk Eisenstadt-Umgebung, Burgenland, 7082, Österreich
Gemeinde Großhöflein, Bezirk Eisenstadt-Umgebung, Burgenland, 7051, Österreich
Hirm, Bezirk Mattersburg, Burgenland, 7024, Österreich
Hornstein/Vorištan, Bezirk Eisenstadt-Umgebung, Burgenland, 7053, Österreich
Gemeinde Jois, Bezirk Neusiedl, Burgenland, 7093, Österreich
Krensdorf, Bezirk Mattersburg, Burgenland, 7031, Österreich
Gemeinde Leithaprodersdorf, Bezirk Eisenstadt-Umgebung, Burgenland, 2443, Österreich
Gemeinde Loretto, Bezirk Eisenstadt-Umgebung, Burgenland, Österreich
Gemeinde Müllendorf, Bezirk Eisenstadt-Umgebung, Burgenland, 7052, Österreich
Neudörfl, Bezirk Mattersburg, Burgenland, 7201, Österreich
Neufeld an der Leitha, Bezirk Eisenstadt-Umgebung, Burgenland, 2491, Österreich
Gemeinde Oggau, Bezirk Eisenstadt-Umgebung, Burgenland, Österreich
Oslip/Uzlop, Bezirk Eisenstadt-Umgebung, Burgenland, 7064, Österreich
Gemeinde Pöttsching, Bezirk Mattersburg, Burgenland, 7033, Österreich
Gemeinde

Neustift bei Schlaining, Bezirk Oberwart, Burgenland, 7433, Österreich
Gemeinde Mariasdorf, Bezirk Oberwart, Burgenland, 7433, Österreich
Gemeinde Oberschützen, Bezirk Oberwart, Burgenland, 7432, Österreich
Gemeinde Wiesfleck, Bezirk Oberwart, Burgenland, 7425, Österreich
Markt Neuhodis, Bezirk Oberwart, Burgenland, 7464, Österreich
Rechnitz, Bezirk Oberwart, Burgenland, 7471, Österreich
Schachendorf, Schachendorf/Čajta, Bezirk Oberwart, Burgenland, 7472, Österreich
Schandorf/Čemba, Bezirk Oberwart, Burgenland, 7472, Österreich
Drumling, Bezirk Oberwart, Burgenland, 7461, Österreich
Goberling, Bezirk Oberwart, Burgenland, 7461, Österreich
Neumarkt im Tauchental, Bezirk Oberwart, Burgenland, 7463, Österreich
Stadtschlaining, Bezirk Oberwart, Burgenland, 7461, Österreich
Gemeinde Unterkohlstätten, Bezirk Oberwart, Burgenland, 7435, Österreich
Allersdorf im Burgenland/Ključarevci, Bezirk Oberwart, Burgenland, 7463, Österreich
Burgauberg-Neudauberg, Güssing, Burgenland, 8291, Österreich
Ha

In [47]:
df_places['Coordinates'] = [x[0] if x else None for x in geoinfo]
df_places['Adresses'] = [x[1] if x else None for x in geoinfo]

In [59]:
df_places.head(30)

,Bundesland,Ort,Coordinates,Adresses
0,Wien,Altmannsdorf,"(16.3217472203092, 48.16443205)","Altmannsdorf, Hoffingergasse, KG Altmannsdorf,..."
1,Wien,Aspern,"(16.4847508, 48.2181854)","Aspern, Donaustadt, Wien, 1220, Österreich"
2,Wien,Atzgersdorf,"(16.2954875, 48.1469838)","Atzgersdorf, Liesing, Wien, 1230, Österreich"
3,Wien,Breitensee,"(16.3064385, 48.2022735)","Breitensee, Penzing, Wien, 1140, Österreich"
4,Wien,Favoriten,"(16.3828218364771, 48.1529937)","Favoriten, Wien, 1100, Österreich"
5,Wien,Gersthof,"(16.3237393, 48.2341382)","Gersthof, Währing, Wien, 1180, Österreich"
6,Wien,Hernals,"(16.2842142186391, 48.2354031)","Hernals, Wien, 1170, Österreich"
7,Wien,Hietzing,"(16.2529858155656, 48.17854095)","Hietzing, Wien, 1130, Österreich"
8,Wien,Inzersdorf,"(16.3527158778638, 48.14397215)","KG Inzersdorf, Liesing, Wien, 1230, Österreich"
9,Wien,Kalksburg,"(16.2457624, 48.1376728)","Kalksburg, Liesing, Wien, 1230, Österreich"


In [60]:
df_places.to_pickle("./df_places.pkl")

In [61]:
df_places = pd.read_pickle("./df_places.pkl")
df_places.head(10)

,Bundesland,Ort,Coordinates,Adresses
0,Wien,Altmannsdorf,"(16.3217472203092, 48.16443205)","Altmannsdorf, Hoffingergasse, KG Altmannsdorf,..."
1,Wien,Aspern,"(16.4847508, 48.2181854)","Aspern, Donaustadt, Wien, 1220, Österreich"
2,Wien,Atzgersdorf,"(16.2954875, 48.1469838)","Atzgersdorf, Liesing, Wien, 1230, Österreich"
3,Wien,Breitensee,"(16.3064385, 48.2022735)","Breitensee, Penzing, Wien, 1140, Österreich"
4,Wien,Favoriten,"(16.3828218364771, 48.1529937)","Favoriten, Wien, 1100, Österreich"
5,Wien,Gersthof,"(16.3237393, 48.2341382)","Gersthof, Währing, Wien, 1180, Österreich"
6,Wien,Hernals,"(16.2842142186391, 48.2354031)","Hernals, Wien, 1170, Österreich"
7,Wien,Hietzing,"(16.2529858155656, 48.17854095)","Hietzing, Wien, 1130, Österreich"
8,Wien,Inzersdorf,"(16.3527158778638, 48.14397215)","KG Inzersdorf, Liesing, Wien, 1230, Österreich"
9,Wien,Kalksburg,"(16.2457624, 48.1376728)","Kalksburg, Liesing, Wien, 1230, Österreich"


In [62]:
df_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2981 entries, 0 to 2980
Data columns (total 4 columns):
Bundesland     2981 non-null object
Ort            2981 non-null object
Coordinates    2952 non-null object
Adresses       2952 non-null object
dtypes: object(4)
memory usage: 93.2+ KB


In [72]:
df_places.Bundesland.unique()

array(['Wien', 'Südtirol', 'Osttiroler', 'Nord tirol', 'Kärnten',
       'Steiermark', 'Salzburg', 'Oberösterreich', 'Niederösterreich',
       'Burgenland'], dtype=object)

In [73]:
df_places[df_places.Coordinates.isnull()]

,Bundesland,Ort,Coordinates,Adresses
215,Osttiroler,Lavant,None,None
290,Nord tirol,Feichten im Kaunertal (Feuchten),None,None
344,Nord tirol,Freundsheim,None,None
345,Nord tirol,Untermieming $Untermiening$,None,None
353,Nord tirol,$Silltall; Anm.: inoffizielle Bezeichnung für ...,None,None
446,Nord tirol,Thiersee $Tiersee$,None,None
484,Nord tirol,Aschau bei Kirchberg,None,None
552,Kärnten,Nölbing,None,None
571,Kärnten,Reisach im Gailtail,None,None
580,Kärnten,Kreuschlach,None,None


Testing with a missing field...

In [76]:
location = geolocator.geocode('Erlakloster , Niederösterreich')
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

AttributeError: 'NoneType' object has no attribute 'address'

Showing places in a map

In [11]:
df_places = pd.read_pickle("./df_places_plus.pkl")
df_places.head(10)

,Bundesland,Ort,Coordinates,Adresses
0,W.,Altmannsdorf,"(10.4342455, 49.9358878)","Altmannsdorf, Gerolzhofen (VGem), Landkreis Sc..."
1,W.,Aspern,"(16.4847508, 48.2181854)","Aspern, Donaustadt, Wien, 1220, Österreich"
2,W.,Atzgersdorf,"(16.2954875, 48.1469838)","Atzgersdorf, Liesing, Wien, 1230, Österreich"
3,W.,Breitensee,"(10.5435061, 50.3531716)","Breitensee, Bad Königshofen i. Grabfeld (VGem)..."
4,W.,Favoriten,"(16.3828218364771, 48.1529937)","Favoriten, Wien, 1100, Österreich"
5,W.,Gersthof,"(16.3237393, 48.2341382)","Gersthof, Währing, Wien, 1180, Österreich"
6,W.,Hernals,"(16.2842142186391, 48.2354031)","Hernals, Wien, 1170, Österreich"
7,W.,Hietzing,"(16.2529858155656, 48.17854095)","Hietzing, Wien, 1130, Österreich"
8,W.,Inzersdorf,"(16.3527158778638, 48.14397215)","KG Inzersdorf, Liesing, Wien, 1230, Österreich"
9,W.,Kalksburg,"(16.2457624, 48.1376728)","Kalksburg, Liesing, Wien, 1230, Österreich"


In [12]:
df_places.Coordinates[1][-1::-1]

(48.2181854, 16.4847508)

In [13]:
#https://ipyleaflet.readthedocs.io/en/latest/
#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
#https://github.com/jupyter-widgets/ipyleaflet/issues/64
#http://andrewgaidus.com/leaflet_webmaps_python/

#layer = WMSLayer(url="https://demo.boundlessgeo.com/geoserver/ows?", layers="nasa:bluemarble")
layer = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-09-30")
center = (47,14)

mapplaces = ipyleaflet.Map(#layers=(layer,),
                            center=center,
                            zoom=6)

mapplaces.layout.width = '100%'
mapplaces.layout.height = '800px'

markers = []
for location in df_places.Coordinates:
    if location:
        markers.append(ipyleaflet.Marker(location=location[-1::-1], draggable=False,))
                       
marker_cluster = ipyleaflet.MarkerCluster(markers=tuple(markers))                       
mapplaces.add_layer(marker_cluster);


mapplaces

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [14]:
#using Folio

mapplaces = folium.Map(location=(47,14), zoom_start=4)

mc = folium.plugins.MarkerCluster()

for location in df_places.Coordinates:
    if location:
        mc.add_child(folium.Marker(location=location[-1::-1]))
                       
mapplaces.add_child(mc)
mapplaces

#### Reading the XML Data files:

In [15]:
xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
#xmlfile = open(xmlfile, mode='r', encoding='utf-8').read()
namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [16]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

Dictionary Approach

In [17]:
xml2dictfile = xmltodict.parse((open(xmlfile, mode='r', encoding='utf-8').read()))

In [18]:
print(xml2dictfile['TEI'].keys())

odict_keys(['@xmlns', 'teiHeader', 'text'])


In [19]:
xml2dictfile['TEI']['@xmlns']
#xml2dictfile['TEI']['teiHeader']

'http://www.tei-c.org/ns/1.0'

In [20]:
print(len(xml2dictfile['TEI']['text']['body']['entry']))
xml2dictfile['TEI']['text']['body']['entry'][0].keys()

1991


odict_keys(['@xml:id', '@xml:lang', 'form', 'gramGrp', 'sense', 'cit', 'note', 'ref', 'etym', 'usg'])

In [76]:
xml2dictfile['TEI']['text']['body']['entry'][144]

OrderedDict([('@xml:id', 'd157_qdb-d1e3912'),
             ('@xml:lang', 'bar'),
             ('form',
              OrderedDict([('@type', 'hauptlemma'), ('orth', 'Ding')])),
             ('gramGrp', OrderedDict([('pos', 'Subst')])),
             ('cit',
              OrderedDict([('@type', 'kontext'),
                           ('@n', '1'),
                           ('quote',
                            ['|E DinN [n] wöi |E Sog',
                             OrderedDict([('@resp', '#JB'),
                                          ('@change', '01'),
                                          ('#text', '|E Dinŋ wöi |E Sog')])]),
                           ('def',
                            OrderedDict([('@xml:lang', 'de'),
                                         ('#text',
                                          'ein Ding wie ein Sack; dickes Kind')])),
                           ('ref',
                            OrderedDict([('@type', 'fragebogenNummer'),
                        

In [22]:
datapath = './TEI-XML-2018/'
onlydirs = [f for f in os.listdir(datapath) if os.path.isdir(os.path.join(datapath,f)) and f.startswith('bel')]
onlydirs

['bel_d-tei-02',
 'bel_e-tei-02',
 'bel_f-tei-02',
 'bel_g-tei-02',
 'bel_h-tei-02',
 'bel_i-j-tei-02',
 'bel_k-tei-02',
 'bel_l-tei-02',
 'bel_m-tei-02',
 'bel_n-o-p-tei-02',
 'bel_q-r-tei-02',
 'bel_s-tei-02',
 'bel_u-w-tei-02',
 'bel_z-tei-02']

In [23]:
onlyfiles = []
for folder in onlydirs:
    for filex in [f for f in os.listdir(os.path.join(datapath, folder)) if os.path.isfile(os.path.join(datapath, folder, f)) \
                                       and not f.startswith('.')]:
        onlyfiles.append('{}{}/{}'.format(datapath, folder, filex))


print('Files in the folder:')
for i, w in enumerate(onlyfiles):
    print(i+1, '--' ,w)

Files in the folder:
1 -- ./TEI-XML-2018/bel_d-tei-02/d154_156_qdb-TEI-02.xml
2 -- ./TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml
3 -- ./TEI-XML-2018/bel_d-tei-02/d158_qdb-TEI-02.xml
4 -- ./TEI-XML-2018/bel_d-tei-02/d159_qdb-TEI-02.xml
5 -- ./TEI-XML-2018/bel_d-tei-02/d160_qdb-TEI-02.xml
6 -- ./TEI-XML-2018/bel_d-tei-02/d161_qdb-TEI-02.xml
7 -- ./TEI-XML-2018/bel_d-tei-02/d162_qdb-TEI-02.xml
8 -- ./TEI-XML-2018/bel_d-tei-02/d163_qdb-TEI-02.xml
9 -- ./TEI-XML-2018/bel_d-tei-02/d164_qdb-TEI-02.xml
10 -- ./TEI-XML-2018/bel_d-tei-02/d165_qdb-TEI-02.xml
11 -- ./TEI-XML-2018/bel_d-tei-02/d166_qdb-TEI-02.xml
12 -- ./TEI-XML-2018/bel_d-tei-02/d167_qdb-TEI-02.xml
13 -- ./TEI-XML-2018/bel_d-tei-02/d168_qdb-TEI-02.xml
14 -- ./TEI-XML-2018/bel_d-tei-02/d169_qdb-TEI-02.xml
15 -- ./TEI-XML-2018/bel_d-tei-02/d170_qdb-TEI-02.xml
16 -- ./TEI-XML-2018/bel_d-tei-02/d171_qdb-TEI-02.xml
17 -- ./TEI-XML-2018/bel_d-tei-02/d172_qdb-TEI-02.xml
18 -- ./TEI-XML-2018/bel_d-tei-02/d173_qdb-TEI-02.xml
19 -- ./TEI-

In [27]:
df_lemmas = pd.DataFrame(columns=['Ort', 'Lemma'])
orts = []
lemmas = []

for idx1, filex in enumerate(onlyfiles):
    xmldictfile = xmltodict.parse((open(filex, encoding="utf-8").read()))
    for idx2, entry in enumerate(xml2dictfile['TEI']['text']['body']['entry']):
        idx = ((idx1+1)*(idx2+1)-1)
        
        ort = None
        if 'usg' in entry.keys():
            if isinstance(entry['usg'], list):
                for local in entry['usg']:
                    try:
                        ort = local['placeName']['#text']
                    except:
                        ort = local['placeName']
            else:
                try:
                    ort = entry['usg']['placeName']['#text']
                except:
                    ort = entry['usg']['placeName']
        orts.append(ort)

        lemma = None
        if 'form' in entry.keys():
            if isinstance(entry['form'], list):
                for elem in entry['form']:
                    if elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], str):          
                        lemma = elem['orth']
                    elif elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], list):
                        for subelem in elem['orth']:
                            if subelem['@type'] == 'normalized':
                                lemma = subelem['#text']
            else:
                if isinstance(entry['form']['orth'], list):
                    for elem in entry['form']['orth']:
                        if elem['@type'] == 'normalized':
                            lemma = elem['#text']
                else:
                    lemma = entry['form']['orth']
        lemmas.append(lemma)
        
        if idx % 10000 == 0:
            print(idx)

df_lemmas['Ort'] = orts
df_lemmas['Lemma'] = lemmas

0
30000
20000
130000
10000
120000
110000
20000
170000
10000
200000
20000
40000
210000
120000
40000
350000
520000
20000
430000
430000
200000
610000
600000
860000
350000
890000
200000
300000
560000
980000
830000
830000
1030000
1070000
120000
870000
760000
590000
300000
980000
1200000
860000
920000
720000


In [28]:
df_lemmas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1451439 entries, 0 to 1451438
Data columns (total 2 columns):
Ort      798984 non-null object
Lemma    1451439 non-null object
dtypes: object(2)
memory usage: 22.1+ MB


In [82]:
df_lemmas.head(40)

,Ort,Lemma
0,Matrei/O. OTir.,Tin
1,Matrei/O. OTir.,Häutin
2,Matrei/O. OTir.,Gesotttin
3,Matrei/O. OTir.,Stadel
4,Matrei/O. OTir.,Tin
5,Matrei/O. OTir.,Stadeltor
6,Salurn,Tina
7,Salurn,Tina
8,(Umg.) Herzogenb-g. NÖ,Dinamít
9,None,Dinamít


In [83]:
df_lemmas = pd.read_pickle("./df_lemmas.pkl")
df_lemmas.head(100)

,Ort,Lemma
0,Matrei/O. OTir.,Tin
1,Matrei/O. OTir.,Häutin
2,Matrei/O. OTir.,Gesotttin
3,Matrei/O. OTir.,Stadel
4,Matrei/O. OTir.,Tin
5,Matrei/O. OTir.,Stadeltor
6,Salurn,Tina
7,Salurn,Tina
8,(Umg.) Herzogenb-g. NÖ,Dinamít
9,None,Dinamít


In [25]:
grouped = df_lemmas.groupby('Ort')['Lemma'].apply(list)

In [26]:
grouped.head()

Ort
"Passeier"                                       [dingen, dingen, dingen, Tinne, dingen, dingen...
"uWaldv." = öWaldv. in NÖ                        [Tinte, Tinte, Tinte, Tinte, Tinte, Tinte, Tin...
(D/Gm./Bz.) Poysdf. NÖ                           [Diras, Diras, Diras, Diras, Diras, Diras, Dir...
(Ebene Reichenau in Gm.) Reichenau Kä.           [Ding, Tingel Tangel, Tinte, sitzen, Piertippl...
(Leimsgrub u. Moarhäuser in Gm.) Schönbg. BöW    [Ding, Ding, Ding, Weile, Ding, Ding, hinterhi...
Name: Lemma, dtype: object

In [27]:
len(grouped)

428

#### Georeferencing Lemmas

In [ ]:
lemma_places = []
for p in df_freqdist.Place:
    lemma_places.append(get_coordinates(p))
    
lemma_places

In [ ]:
idx = df_freqdist.Place.index
idx

In [31]:
with open('./df_freqdist.pkl', 'rb') as f:
    df_freqdist = pickle.load(f)

df_freqdist.head()

,Place,Coordinates,Adresses
0,Matrei/O. OTir.,"(11.4489464, 47.122137)","Matrei, Brenner Autobahn, Mühlbachl, Innsbruck..."
1,Salurn,"(11.2110749, 46.2396083)","Salorno - Salurn, Überetsch-Unterland - Oltrad..."
2,(Umg.) Herzogenb-g. NÖ,"(23.3536536859833, 42.66048145)","МГУ, проф. Боян Каменов, район Студентски, Сто..."
3,Königswerth Egerld.,"(12.6803156, 50.1938078)","Královské Poříčí, okres Sokolov, Karlovarský k..."
4,Marling,"(11.1402909, 46.6555608)","Marling - Marlengo, Burggrafenamt - Burgraviat..."


In [46]:
with open('./df_joined.pkl', 'rb') as f:
    df_joined = pickle.load(f)

df_joined.head(10)

,Coordinates,Adresses,Lemma
Place,,,
Matrei/O. OTir.,"(11.4489464, 47.122137)","Matrei, Brenner Autobahn, Mühlbachl, Innsbruck...","[Tin, Häutin, Gesotttin, Stadel, Tin, Stadelto..."
Salurn,"(11.2110749, 46.2396083)","Salorno - Salurn, Überetsch-Unterland - Oltrad...","[Tina, Tina, Tina, Tina, Tina, Tina, Tina, Tin..."
(Umg.) Herzogenb-g. NÖ,"(23.3536536859833, 42.66048145)","МГУ, проф. Боян Каменов, район Студентски, Сто...","[Dinamít, Dinamo, Dinastí, dinástisch, Ding, D..."
Königswerth Egerld.,"(12.6803156, 50.1938078)","Královské Poříčí, okres Sokolov, Karlovarský k...","[Dinamo, Ding, Ding, Gedinge, Gedinge, Gedinge..."
Marling,"(11.1402909, 46.6555608)","Marling - Marlengo, Burggrafenamt - Burgraviat...","[Tina, Tina, Tina, Tina, Tina, Tina, Tina, Tin..."
Malta Kä.,"(13.5041576, 46.9558603)","Malta, Spittal an der Drau, Kärnten, 9854, Öst...","[tindern, tindern, schebern, scheppern, tusche..."
Wien,"(16.3725042, 48.2083537)","Wien, 1010, Österreich","[Tinef, Tinef, Tinef, Schmarren, Drëck, Ding, ..."
Böwald,"(-89.2770777, 40.7266249)","Bowald Street, Eureka, Woodford County, Illino...","[Tinellein, Tinellein, Tinellein, Tinellein, T..."
STir.,"(25.6421104, 51.1141019)","Стир, Маневицький район, Волинська область, 35...","[tinen, Fürdinger, Tintinger, tinseln, Tiritar..."


In [47]:
df_joined.reset_index(inplace=True)

In [48]:
df_joined.head(10)

,Place,Coordinates,Adresses,Lemma
0,Matrei/O. OTir.,"(11.4489464, 47.122137)","Matrei, Brenner Autobahn, Mühlbachl, Innsbruck...","[Tin, Häutin, Gesotttin, Stadel, Tin, Stadelto..."
1,Salurn,"(11.2110749, 46.2396083)","Salorno - Salurn, Überetsch-Unterland - Oltrad...","[Tina, Tina, Tina, Tina, Tina, Tina, Tina, Tin..."
2,(Umg.) Herzogenb-g. NÖ,"(23.3536536859833, 42.66048145)","МГУ, проф. Боян Каменов, район Студентски, Сто...","[Dinamít, Dinamo, Dinastí, dinástisch, Ding, D..."
3,Königswerth Egerld.,"(12.6803156, 50.1938078)","Královské Poříčí, okres Sokolov, Karlovarský k...","[Dinamo, Ding, Ding, Gedinge, Gedinge, Gedinge..."
4,Marling,"(11.1402909, 46.6555608)","Marling - Marlengo, Burggrafenamt - Burgraviat...","[Tina, Tina, Tina, Tina, Tina, Tina, Tina, Tin..."
5,Malta Kä.,"(13.5041576, 46.9558603)","Malta, Spittal an der Drau, Kärnten, 9854, Öst...","[tindern, tindern, schebern, scheppern, tusche..."
6,Wien,"(16.3725042, 48.2083537)","Wien, 1010, Österreich","[Tinef, Tinef, Tinef, Schmarren, Drëck, Ding, ..."
7,Böwald,"(-89.2770777, 40.7266249)","Bowald Street, Eureka, Woodford County, Illino...","[Tinellein, Tinellein, Tinellein, Tinellein, T..."
8,STir.,"(25.6421104, 51.1141019)","Стир, Маневицький район, Волинська область, 35...","[tinen, Fürdinger, Tintinger, tinseln, Tiritar..."
9,alt in Laurein,"(24.4628189399597, 44.29050505)","Olt, România","[tinern, tinern, tinern, tinern, tinern, tiner..."


In [81]:
df_joined.to_csv('./df_joined.csv')

In [73]:
def list2string(lista):
    texto = ''
    for elem in lista:
        texto = texto + '\n' + elem[0] + ' --- ' + str(elem[1]) + '\n'
    return texto

In [72]:
df_joined.Lemma

0      [Tin, Häutin, Gesotttin, Stadel, Tin, Stadelto...
1      [Tina, Tina, Tina, Tina, Tina, Tina, Tina, Tin...
2      [Dinamít, Dinamo, Dinastí, dinástisch, Ding, D...
3      [Dinamo, Ding, Ding, Gedinge, Gedinge, Gedinge...
4      [Tina, Tina, Tina, Tina, Tina, Tina, Tina, Tin...
5      [tindern, tindern, schebern, scheppern, tusche...
6      [Tinef, Tinef, Tinef, Schmarren, Drëck, Ding, ...
7      [Tinellein, Tinellein, Tinellein, Tinellein, T...
8      [tinen, Fürdinger, Tintinger, tinseln, Tiritar...
9      [tinern, tinern, tinern, tinern, tinern, tiner...
10     [Ding, Ding, Ding, Ding, Ding, Ding, Ding, Din...
11     [Ding, Ding, Ding, Ding, Ding, Ding, Ding, Din...
12     [Mäcke, Mäcke, Mäcke, Mäcke, Mäcke, Mäcke, Mäc...
13     [Ding, Leibgedinge, Leibgedinge, andiníeren, d...
14     [Ding, Ding, guotding, guotding, Dingerling, G...
15     [Ding, guotding, dingen, dingen, ausbedingen, ...
16     [Ding, Ding, Ding, Ding, Ding, Ding, Ding, Din...
17     [Ding, Ding, Ding, Tinte

In [69]:
center = (47,14)
mapplaces = ipyleaflet.Map(#layers=(layer,),
                            center=center,
                            zoom=6)

mapplaces.layout.width = '100%'
mapplaces.layout.height = '800px'

markers = []
for idx in df_joined.index:
    if df_joined.loc[idx,'Coordinates'] and not isinstance(df_joined.loc[idx,'Lemma'], float):
        title = list2string(collections.Counter(df_joined.loc[idx,'Lemma']).most_common(6))
        markers.append(ipyleaflet.Marker(location=df_joined.loc[idx,'Coordinates'][-1::-1], draggable=False, title=title))
                       
marker_cluster = ipyleaflet.MarkerCluster(markers=tuple(markers))                       
mapplaces.add_layer(marker_cluster);


mapplaces

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [74]:
#using Folio

mapplaces = folium.Map(location=(47,14),
               zoom_start=4)

mc = folium.plugins.MarkerCluster()

for idx in df_joined.index:
     if df_joined.loc[idx,'Coordinates'] and not isinstance(df_joined.loc[idx,'Lemma'], float):
        title = list2string(collections.Counter(df_joined.loc[idx,'Lemma']).most_common(6))
        mc.add_child(folium.Marker(location=df_joined.loc[idx,'Coordinates'][-1::-1], popup=title))
        
                     
mapplaces.add_child(mc)
mapplaces

In [75]:
mapplaces.save('./lemmas_places.html')